These are the libraries that we use.

In [1]:
import os
import re
import operator
import warnings
import joblib

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

import numpy as np
import math

import nltk
nltk.download('all')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.utils.extmath import cartesian

from IPython.display import SVG, clear_output
from graphviz import Source
from IPython.display import display

tences to
[nltk_data]    |     C:\Users\Michelangelo\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package comparative_sentences is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package comtrans to
[nltk_data]    |     C:\Users\Michelangelo\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package comtrans is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\Michelangelo\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\Michelangelo\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package conll2007 to
[nltk_data]    |     C:\Users\Michelangelo\AppData\Roaming\nltk_data..
[nltk_data]    |     .
[nltk_data]    |   Package conll2007 is alread

ModuleNotFoundError: No module named 'graphviz'

Here we create the dataset we're going to use to train our Binary Decision Tree model. We make sure that both accepted and rejected tites are not duplicated. We get all the accepted titles (which are way fewer than the rejected ones), and then we get a sample of the rejected ones big enough as to make the union between accepted and rejected contain 1000 samples.

In [2]:
df = pd.read_csv('scopus_search_results.csv', encoding='UTF-8')
acceptedDataSet = df[['Title','Accepted']][df['Accepted'] == True]
acceptedDataSet = acceptedDataSet.drop_duplicates()
# since we have many rejected titles, get a random sample among them 
# so that the union between accepted and rejected is 1000 titles
rejectedDataSet = df[['Title','Accepted']][df['Accepted'] == False].drop_duplicates().sample(n=1000-len(acceptedDataSet), random_state=1)
joinDataset = pd.concat([acceptedDataSet,rejectedDataSet],ignore_index=True)
print(len(acceptedDataSet))
print(len(rejectedDataSet))
print(len(joinDataset))
# print(joinDataset)


330
670
1000


Titles preprocessing:
 - remove special and single characters
 - trim whitespaces
 - convert to lowercase
 - lemmatize the words: return the root of the words E.G. studying -> study

In [3]:
titles = joinDataset.Title
classifications = joinDataset.Accepted
lemmatizer = WordNetLemmatizer()


def titlePreprocessing(titles):
    preProcessedTitles = []

    for title in titles:
        # Remove all the special characters
        title = re.sub(r'\W', ' ', title)
        
        # remove all single characters
        title = re.sub(r'\s+[a-zA-Z]\s+', ' ', title)
        
        # Remove single characters from the start
        title = re.sub(r'\^[a-zA-Z]\s+', ' ', title) 
        
        # Substituting multiple spaces with single space
        title = re.sub(r'\s+', ' ', title, flags=re.I)
        
        # Removing prefixed 'b'
        title = re.sub(r'^b\s+', '', title)
        
        # Converting to Lowercase
        title = title.lower()
        
        # Lemmatization
        title = title.split()
        
        title = [lemmatizer.lemmatize(word) for word in title]
        title = ' '.join(title)
        
        preProcessedTitles.append(title)
    return preProcessedTitles

preProcessedTitles = titlePreprocessing(titles)
for i in range(0,10):
    print(preProcessedTitles[i])


making best use of model evaluation to compute sensitivity index
controlling individual agent in high density crowd simulation
verification and validation of simulation model
hierarchical model for real time simulation of virtual human crowd
a comprehensive framework for verification validation and uncertainty quantification in scientific computing
practical bayesian model evaluation using leave one out cross validation and waic
transfer of multivariate calibration model review
validation verification and testing of computer software
a time domain approach to model validation
model validation connection between robust control and identification


Split the dataset into train and test and convert the classes into binary values because that's a requirement to fit a Binary Decision Tree.

In [4]:
train_titles, test_titles, train_classes, test_classes = train_test_split(preProcessedTitles, classifications, test_size=0.2, random_state=1)

# Classes must be binary to train a Binary Decision Tree so False -> 0 and True -> 1
train_classes = train_classes.astype(int)
test_classes = test_classes.astype(int)

print(train_classes.value_counts())
print(test_classes.value_counts())

0    535
1    265
Name: Accepted, dtype: int64
0    135
1     65
Name: Accepted, dtype: int64


Train the Binary Decision Tree on the train set.

In [5]:
vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern = r'\b[\w]{4,}(?<![\d])\b', stop_words=stopwords.words('english'), min_df = 2)

# Create Decision Tree classifer object
max_depth = 11
min_samples_split = 0.015789474
min_samples_leaf = 0.00526315789473684
criterion="gini"
class_weight = "balanced"
splitter = "best"

dtc = DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, criterion=criterion, class_weight = class_weight, splitter = splitter)
# dtc = DecisionTreeClassifier(criterion="entropy", splitter = "best", max_depth = 5, min_samples_leaf = 5, class_weight = "balanced")

dtc_pipe = Pipeline([('vectorizer', vectorizer), ('dtc', dtc)])

# Train Decision Tree Classifer
dtc_pipe.fit(train_titles,train_classes)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(min_df=2, ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 token_pattern='\\b[\\w]{4,}(?<![\\d])\\b')),
                ('dtc',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=11,
                                        min_samples_leaf=0.00526315789473684,
                           

Test the accuracy of the prediction on the test set.

In [6]:
#Predict the response for test dataset
predicted_classes = dtc_pipe.predict(test_titles)

print(classification_report(test_classes, predicted_classes))
print(pd.crosstab(test_classes, predicted_classes, rownames=['True'], colnames=['Predicted'], margins=True))
print("Accuracy:",accuracy_score(test_classes, predicted_classes))

test_recall_score = recall_score(test_classes, predicted_classes, average='binary',pos_label = 1) 
print("test_recall_score: %s"%test_recall_score)

precision    recall  f1-score   support

           0       0.91      0.87      0.89       135
           1       0.75      0.82      0.78        65

    accuracy                           0.85       200
   macro avg       0.83      0.84      0.83       200
weighted avg       0.85      0.85      0.85       200

Predicted    0   1  All
True                   
0          117  18  135
1           12  53   65
All        129  71  200
Accuracy: 0.85
test_recall_score: 0.8153846153846154


Test the accuracy of the cross validation.

In [7]:
classifications = classifications.astype(int)
scores = cross_validate(dtc_pipe, titles, classifications, cv=10, scoring = ('recall', 'precision', 'roc_auc'))

print("recall cross validation: %0.2f (+/- %0.2f)" % (scores['test_recall'].mean(), scores['test_recall'].std() * 2))
print("test_precision cross validation: %0.2f (+/- %0.2f)" % (scores['test_precision'].mean(), scores['test_precision'].std() * 2))
print("test_roc_auc cross validation: %0.2f (+/- %0.2f)" % (scores['test_roc_auc'].mean(), scores['test_roc_auc'].std() * 2))

recall cross validation: 0.77 (+/- 0.08)
test_precision cross validation: 0.76 (+/- 0.12)
test_roc_auc cross validation: 0.88 (+/- 0.06)


This is to visualize the resulting Binary Decision Tree Model. It requires "graphviz" (pip install graphviz)

In [ ]:
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'

graph = Source(
    tree.export_graphviz(
        dtc,
        out_file=None,
        feature_names=vectorizer.get_feature_names(),
        class_names=['1', '0'],
        filled = True)
)

savePath = ".\models\\renders\\"
graph.format = 'pdf'
graph.render(str('%sdtc_maxdepth_%s_minsampleleaf_%s'%(savePath, max_depth,min_samples_leaf)),view=True)

Parameter Fitting: we try to get the best possible parameters for our Binary Decision Tree Classifier by bruteforcing and plotting the results.

In [ ]:
max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
   dtc = DecisionTreeClassifier(max_depth=max_depth)
   dtc_pipe = Pipeline([('vectorizer', vectorizer), ('dtc', dtc)])
   dtc_pipe.fit(train_titles,train_classes)

   train_pred = dtc_pipe.predict(train_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(train_classes, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous train results
   train_results.append(roc_auc)
   y_pred = dtc_pipe.predict(test_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(test_classes, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous test results
   test_results.append(roc_auc)

line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Tree depth")
# plt.show()
plt.savefig("tree_depth_optimization.pdf", format="pdf")

In [ ]:
min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
train_results = []
test_results = []
for min_samples_split in min_samples_splits:
   dtc = DecisionTreeClassifier(min_samples_split=min_samples_split)
   dtc_pipe = Pipeline([('vectorizer', vectorizer), ('dtc', dtc)])
   dtc_pipe.fit(train_titles,train_classes)

   train_pred = dtc_pipe.predict(train_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(train_classes, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous train results
   train_results.append(roc_auc)
   y_pred = dtc_pipe.predict(test_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(test_classes, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous test results
   test_results.append(roc_auc)

line1, = plt.plot(min_samples_splits, train_results, 'b', label="Train AUC")
line2, = plt.plot(min_samples_splits, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Min Samples Split")
# plt.show()
plt.savefig("min_samples_splits_optimization.pdf", format="pdf")


In [ ]:
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
train_results = []
test_results = []
for min_samples_leaf in min_samples_leafs:
   dtc = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
   dtc_pipe = Pipeline([('vectorizer', vectorizer), ('dtc', dtc)])
   dtc_pipe.fit(train_titles,train_classes)

   train_pred = dtc_pipe.predict(train_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(train_classes, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous train results
   train_results.append(roc_auc)
   y_pred = dtc_pipe.predict(test_titles)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(test_classes, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous test results
   test_results.append(roc_auc)

line1, = plt.plot(min_samples_leafs, train_results, 'b', label="Train AUC")
line2, = plt.plot(min_samples_leafs, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("min samples leaf")
# plt.show()
plt.savefig("min_samples_leafs_splits_optimization.pdf", format="pdf")

Progress Bar Function

In [ ]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

Find the best combination of parameters.

In [ ]:
warnings.filterwarnings('ignore')

max_depths = np.linspace(7, 12, 5, endpoint=True)
min_samples_splits = np.linspace(np.nextafter(0,1), 0.1, 20, endpoint=True)
min_samples_leafs = np.linspace(np.nextafter(0,1), 0.1, 20, endpoint=True)
criterion="gini"
class_weight = "balanced"
splitter = "best"

paramCombinations = cartesian([max_depths, min_samples_splits, min_samples_leafs])

data = []
columns = ['max_depth', 'min_samples_split', 'min_samples_leaf', 'train_accuracy_score', 'train_f1_score', 'train_recall_score', 'train_roc_auc_score', 'test_accuracy_score','test_f1_score','test_recall_score','test_roc_auc_score']
count = 0
for pComb in paramCombinations:
    printProgressBar(count, len(paramCombinations), prefix = 'Progress:', suffix = 'Complete', length = 50)
    # print advancement percentage
    # if(((count/len(paramCombinations)*100) % 1) == 0): 
    #     clear_output()
    #     print(str("%d%%"%(count/len(paramCombinations)*100)))
    
    dtc = DecisionTreeClassifier(max_depth = pComb[0], min_samples_split = pComb[1], min_samples_leaf = pComb[2], 
        criterion=criterion,splitter=splitter,class_weight=class_weight)
    dtc_pipe = Pipeline([('vectorizer', vectorizer), ('dtc', dtc)])
    dtc_pipe.fit(train_titles,train_classes)

    train_pred = dtc_pipe.predict(train_titles)
    
    train_accuracy_score = accuracy_score(train_classes, train_pred)
    train_f1_score = f1_score(train_classes, train_pred, average='weighted')
    # train_recall_score = recall_score(train_classes, train_pred, average='weighted') 
    # train_f1_score = f1_score(train_classes, train_pred, average='binary',pos_label = 1)
    train_recall_score = recall_score(train_classes, train_pred, average='binary',pos_label = 1) 
    train_roc_auc_score = roc_auc_score(train_classes, train_pred)
    
    test_pred = dtc_pipe.predict(test_titles)

    test_accuracy_score = accuracy_score(test_classes, test_pred)
    test_f1_score = f1_score(test_classes, test_pred, average='weighted')
    # test_recall_score = recall_score(test_classes, test_pred, average='weighted') 
    # test_f1_score = f1_score(test_classes, test_pred, average='binary',pos_label = 1)
    test_recall_score = recall_score(test_classes, test_pred, average='binary',pos_label = 1) 
    test_roc_auc_score = roc_auc_score(test_classes, test_pred)

    data.append((pComb[0],pComb[1],pComb[2],train_accuracy_score, train_f1_score, train_recall_score,train_roc_auc_score,
        test_accuracy_score, test_f1_score, test_recall_score,test_roc_auc_score))
    count += 1

# create DataFrame using data 
paramCombinationsDataFrame = pd.DataFrame(data, columns = columns)
paramCombinationsDataFrame.to_csv('models/parameters/bruteforce_param_combinations.csv', index=False, header=True)

In [ ]:
print("#Max Depts: %s"%len(max_depths))
print("Max Depts: %s"%max_depths)

print("#Min Samples Splits: %s"%len(min_samples_splits))
print("Min Samples Splits: %s"%min_samples_splits)

print("#Min Samples Leaves: %s"%len(min_samples_leafs))
print("Min Samples Leaves: %s"%min_samples_leafs)

bestTrainIndex, bestTrainRes = max(enumerate(train_results), key=operator.itemgetter(1))
# bestTrainRes = max(train_results)
# bestTrainIndex = train_results.index(max(train_results))
bestTrainParamCombination = paramCombinations[bestTrainIndex]
print(str("Training Set Best Combination: %s Index: %d Score: %f"%(bestTrainParamCombination,bestTrainIndex,bestTrainRes)))

bestTestIndex, bestTestRes = max(enumerate(test_results), key=operator.itemgetter(1))
# bestTestRes = max(test_results)
# bestTestIndex = test_results.index(max(test_results))
bestTestParamCombination = paramCombinations[bestTestIndex]
print(str("Test Set Best Combination: %s Index: %d Score: %f"%(bestTestParamCombination,bestTestIndex,bestTestRes)))

Save the model so that it can later be re-imported.

In [ ]:
savePath = ".\models\\"
extension = ".sav"
filename = str('%sdtc_maxdepth_%s_minsampleleaf_%s%s'%(savePath, max_depth,min_samples_leaf,extension))
joblib.dump(dtc_pipe, filename)

Use the model to predict stuff.

In [ ]:
savePath = ".\models\\"
extension = ".sav"
filename = str('%sdtc_maxdepth_%s_minsampleleaf_%s%s'%(savePath, max_depth,min_samples_leaf,extension))

model = joblib.load(filename)

unreadTitles = df['Title'][df['ReadTimeStamp'].isnull()].drop_duplicates()

preprocessedTitles = titlePreprocessing(unreadTitles)

predictedClasses = model.predict(preprocessedTitles)

class0Titles = set()
class1Titles = set()

for i in range(0,len(predictedClasses),1):
    if(predictedClasses.item(i) == 1):
        class1Titles.add(unreadTitles.iloc[i])
    else:
        class0Titles.add(unreadTitles.iloc[i])

Print the interesting titles.

In [ ]:
for class1Title in class1Titles:
    print(class1Title)
print("\n--- CLASSIFIER:  %s   ---" %filename)
print("---   %d interesting titles found out of %d   ---" %(len(class1Titles), len(unreadTitles)))

Export the interesting articles into a separate CSV file.

In [ ]:
unreadArticles = df[df['ReadTimeStamp'].isnull()]
indexes = set(unreadArticles.index)

class0Rows = set()
class1Rows = set()
for index, article in unreadArticles.iterrows():
    if((article['Title'] in class1Titles) == False):
        class0Rows.add(index)

class1Rows = indexes.difference(class0Rows)

class0Articles = unreadArticles.drop(class1Rows)
class1Articles = unreadArticles.drop(class0Rows)

print(unreadArticles.shape[0])
print("%d + %d = %d"%(class0Articles.shape[0],class1Articles.shape[0], class0Articles.shape[0]+class1Articles.shape[0]))

class0Articles.to_csv('class_0_articles.csv', index=False, header=True)
class1Articles.to_csv('class_1_articles.csv', index=False, header=True)
